In [1]:
import pandas as pd
import numpy as np
import os
import networkx as nx
from pyvis.network import Network
from sklearn.preprocessing import MinMaxScaler

In [2]:
def draw_graph(country_file):
    entity_wel_df = pd.read_feather(data_path+"entity_weighted_edgelist_ALL_"+country_file+".feather")
    # import edgelist data
    # entity_wel_df = pd.read_csv('entity_weighted_edgelist_ML.csv', index_col=0)
    entity_wel_df['ent_rel_pair'] = entity_wel_df['head_text'] +' '+entity_wel_df['relation']+' '+entity_wel_df['tail_text']
    entity_wel_df['head_name'] = entity_wel_df['head_text']+'_'+entity_wel_df['head_wiki_id']
    entity_wel_df['tail_name'] = entity_wel_df['tail_text']+'_'+entity_wel_df['tail_wiki_id']

    # node size
    head_node_weight_df = entity_wel_df.groupby(['head_name']).sum().sort_values('count', ascending=False).reset_index()
    tail_node_weight_df = entity_wel_df.groupby(['tail_name']).sum().sort_values('count', ascending=False).reset_index()
    head_node_weight_df = entity_wel_df.merge(head_node_weight_df, left_on=['head_name'], right_on=['head_name'], how='right')
    tail_node_weight_df = entity_wel_df.merge(tail_node_weight_df, left_on=['tail_name'], right_on=['tail_name'], how='right')
    head_node_weight_df['node'] = head_node_weight_df['head_text']
    head_node_weight_df['size'] = head_node_weight_df['count_y']
    head_node_weight_df = head_node_weight_df[['size','node']]
    tail_node_weight_df['node'] = tail_node_weight_df['tail_text']
    tail_node_weight_df['size'] = tail_node_weight_df['count_y']
    tail_node_weight_df = tail_node_weight_df[['size','node']]
    node_weight_df = pd.concat([head_node_weight_df, tail_node_weight_df])
    node_weight_df['size'] = MinMaxScaler(feature_range=(10, 50)).fit_transform(np.array(node_weight_df['size']).reshape(-1,1))
    
    
    
    # df = entity_wel_df.iloc[:1000]
    mention_num = 1
    # entity_wel_df = entity_wel_df[entity_wel_df['head_text'] != 'Elsevier'] # df = df[~df[col].isin([string])]
    # entity_wel_df = entity_wel_df[entity_wel_df['tail_text'] != 'Elsevier']
    df = entity_wel_df[entity_wel_df['count']>mention_num]
    df.rename(columns={"count": "value", "relation":"title"}, inplace=True)
    node_lst = df['head_text'].tolist()+df['tail_text'].tolist()
    node_weight_df = node_weight_df.drop_duplicates(subset='node').reset_index(drop=True)
    node_weight_df = node_weight_df[node_weight_df['node'].isin(node_lst)]
    node_attr = node_weight_df.set_index('node').to_dict('index')
    G = nx.from_pandas_edgelist(df, source='head_text', target='tail_text', edge_attr=['value', 'title'])
    nx.set_node_attributes(G, node_attr)

    nt = Network(height="1000px", width="100%", notebook=True,directed=True)
    # populates the nodes and edges data structures
    nt.from_nx(G)
    nt.toggle_physics(True)
    nt.show('Graph_network_'+country_file+'.html')
    node_weight_df.to_csv('node_weight_'+country_file+'.csv', index=False)
    entity_wel_df.to_csv('edge_data_'+country_file+'.csv', index=False)
    print("Done "+country_file+'\n')

In [3]:
pwd = os.getcwd()
data_path = os.path.dirname(pwd) + "\\Entity_edgelist\\"
file_lst = ['Bangladesh MDP', 'Bangladesh NDP', 'Bangladesh VNR',
       'Botswana NDP', 'Botswana VNR', 'Cameroon MDP',
       'Cameroon NDP', 'Eswatini NDP', 'Eswatini VNR',
       'Gambia NDP', 'Gambia VNR', 'Ghana MDP',
       'Ghana NDP', 'Ghana VNR', 'Kenya MDP', 'Kenya NDP',
       'Kenya VNR', 'Lao NDP', 'Lao VNR', 'Liberia MDP',
       'Liberia NDP', 'Liberia VNR', 'Malawi NDP',
       'Malawi VNR', 'Namibia MDP', 'Namibia NDP',
       'Namibia VNR', 'Nigeria NDP', 'Nigeria VNR',
       'Pakistan NDP', 'Pakistan VNR', 'Rwanda NDP',
       'Rwanda VNR', 'South Africa NDP', 'South Africa VNR',
       'Soutn Africa MDP', 'Sri Lanka NDP', 'Sri Lanka VNR',
       'Tanzania MDP', 'Tanzania NDP', 'Togo NDP',
       'Togo VNR', 'Zambia NDP', 'Zambia VNR',
       'Zimbabwe NDP', 'Zimbabwe VNR']
for i in file_lst:
    try:
        draw_graph(i)
    except:
        print(i," DOES NOT WORK.")

C:\Users\gcmar\AppData\Local\Temp\ipykernel_84684\2363068920.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.rename(columns={"count": "value", "relation":"title"}, inplace=True)


Local cdn resources have problems on chrome/safari when used in jupyter-notebook. 
Done Bangladesh MDP



C:\Users\gcmar\AppData\Local\Temp\ipykernel_84684\2363068920.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.rename(columns={"count": "value", "relation":"title"}, inplace=True)


Local cdn resources have problems on chrome/safari when used in jupyter-notebook. 
Done Bangladesh NDP



C:\Users\gcmar\AppData\Local\Temp\ipykernel_84684\2363068920.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.rename(columns={"count": "value", "relation":"title"}, inplace=True)


...

Local cdn resources have problems on chrome/safari when used in jupyter-notebook. 
Done Liberia VNR



C:\Users\gcmar\AppData\Local\Temp\ipykernel_84684\2363068920.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.rename(columns={"count": "value", "relation":"title"}, inplace=True)


Local cdn resources have problems on chrome/safari when used in jupyter-notebook. 
Done Malawi NDP



C:\Users\gcmar\AppData\Local\Temp\ipykernel_84684\2363068920.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.rename(columns={"count": "value", "relation":"title"}, inplace=True)


Local cdn resources have problems on chrome/safari when used in jupyter-notebook. 
Done Malawi VNR

Namibia MDP  DOES NOT WORK.
Namibia NDP  DOES NOT WORK.
Namibia VNR  DOES NOT WORK.
Nigeria NDP  DOES NOT WORK.
Nigeria VNR  DOES NOT WORK.
Pakistan NDP  DOES NOT WORK.
Pakistan VNR  DOES NOT WORK.
Rwanda NDP  DOES NOT WORK.
Rwanda VNR  DOES NOT WORK.
South Africa NDP  DOES NOT WORK.
South Africa VNR  DOES NOT WORK.
Soutn Africa MDP  DOES NOT WORK.
Sri Lanka NDP  DOES NOT WORK.
Sri Lanka VNR  DOES NOT WORK.
Tanzania MDP  DOES NOT WORK.
Tanzania NDP  DOES NOT WORK.
Togo NDP  DOES NOT WORK.
Togo VNR  DOES NOT WORK.
Zambia NDP  DOES NOT WORK.
Zambia VNR  DOES NOT WORK.
Zimbabwe NDP  DOES NOT WORK.
Zimbabwe VNR  DOES NOT WORK.


In [4]:
# # import edgelist data
# # entity_wel_df = pd.read_csv('entity_weighted_edgelist_ML.csv', index_col=0)
# entity_wel_df['ent_rel_pair'] = entity_wel_df['head_text'] +' '+entity_wel_df['relation']+' '+entity_wel_df['tail_text']
# entity_wel_df['head_name'] = entity_wel_df['head_text']+'_'+entity_wel_df['head_wiki_id']
# entity_wel_df['tail_name'] = entity_wel_df['tail_text']+'_'+entity_wel_df['tail_wiki_id']

# # node size
# head_node_weight_df = entity_wel_df.groupby(['head_name']).sum().sort_values('count', ascending=False).reset_index()
# tail_node_weight_df = entity_wel_df.groupby(['tail_name']).sum().sort_values('count', ascending=False).reset_index()
# head_node_weight_df = entity_wel_df.merge(head_node_weight_df, left_on=['head_name'], right_on=['head_name'], how='right')
# tail_node_weight_df = entity_wel_df.merge(tail_node_weight_df, left_on=['tail_name'], right_on=['tail_name'], how='right')
# head_node_weight_df['node'] = head_node_weight_df['head_text']
# head_node_weight_df['size'] = head_node_weight_df['count_y']
# head_node_weight_df = head_node_weight_df[['size','node']]
# tail_node_weight_df['node'] = tail_node_weight_df['tail_text']
# tail_node_weight_df['size'] = tail_node_weight_df['count_y']
# tail_node_weight_df = tail_node_weight_df[['size','node']]
# node_weight_df = pd.concat([head_node_weight_df, tail_node_weight_df])
# node_weight_df['size'] = MinMaxScaler(feature_range=(10, 100)).fit_transform(np.array(node_weight_df['size']).reshape(-1,1))
# node_weight_df

In [5]:
# # df = entity_wel_df.iloc[:1000]
# mention_num = 2
# # entity_wel_df = entity_wel_df[entity_wel_df['head_text'] != 'Elsevier'] # df = df[~df[col].isin([string])]
# # entity_wel_df = entity_wel_df[entity_wel_df['tail_text'] != 'Elsevier']
# df = entity_wel_df[entity_wel_df['count']>mention_num]
# df.rename(columns={"count": "value", "relation":"title"}, inplace=True)
# node_lst = df['head_text'].tolist()+df['tail_text'].tolist()
# node_weight_df = node_weight_df.drop_duplicates(subset='node').reset_index(drop=True)
# node_weight_df = node_weight_df[node_weight_df['node'].isin(node_lst)]
# node_attr = node_weight_df.set_index('node').to_dict('index')
# G = nx.from_pandas_edgelist(df, source='head_text', target='tail_text', edge_attr=['value', 'title'])
# nx.set_node_attributes(G, node_attr)

# nt = Network(height="1000px", width="100%", notebook=True,directed=True)
# # populates the nodes and edges data structures
# nt.from_nx(G)
# nt.toggle_physics(True)
# nt.show('ML_above2edges.html')